In [1]:
import ee
ee.Initialize()
import geemap

Map = geemap.Map()

In [2]:
# DATASETS
ls5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
s1 = ee.ImageCollection('COPERNICUS/S1_GRD')
s2 = ee.ImageCollection("COPERNICUS/S2_SR")
admin = ee.FeatureCollection("FAO/GAUL/2015/level0")
rgbVis_LS5 = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['SR_B3', 'SR_B2', 'SR_B1'],
}
rgbVis_S2 = {
  'min': 0.0,
  'max': 0.3,
  'bands': ['B4', 'B3', 'B2'],
}

In [3]:
# REDUCE REGIONS FUNCTION
def reduce_region(image, reducer, geometry, scale):
    reduced = image.reduceRegion(**{
        'reducer': reducer,
        'geometry': geometry,
        'scale': scale
    })

    return reduced

In [4]:
# variables
# size = 0.02
# big mine region (musonoi)
region = ee.Geometry.Polygon([[[25.34, -10.78], [25.34, -10.675], [25.475, -10.675], [25.475, -10.78]]])
# ray mine region (kibali)
#region = ee.Geometry.Polygon([[[29.56, 3.1], [29.56, 3.155], [29.615, 3.155], [29.615, 3.1]]])
# rishi mine region (Kanunka Village)
#region = ee.Geometry.Polygon([[[27.355, -7.56], [27.355, -7.51], [27.415, -7.51], [27.415, -7.56]]])

In [5]:
# CLOUD MASKING
# landsat
def mask_ls5_clouds(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)

    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)

    return image.addBands(opticalBands, None, True) \
            .updateMask(qaMask) \
            .updateMask(saturationMask)

# sentinel
def mask_s2_clouds(image):
    qa = image.select('QA60')

    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))

    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start"])

In [6]:
# TRAINING DATA & CLASSIFIERS
training_region = ee.Geometry.Polygon(
        [[[29.554129272985683, 3.1591674847348235],
          [29.554129272985683, 3.092319151883147],
          [29.625197083044277, 3.092319151883147],
          [29.625197083044277, 3.1591674847348235]]])

# landsat
bare = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.566649352977876, 3.131936594576548]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.594069479589553, 3.143136886698536]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.578229613742543, 3.109209943820683]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.581158585986806, 3.1067030888196494]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.55884031463487, 3.11517956941471]),
            {
              "landcover": 0,
              "system:index": "4"
            })])

vegetation = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.581819366025293, 3.132088091449441]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.595938514279688, 3.131445322675145]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.59962923388418, 3.1347448648629315]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.565039466428125, 3.104877202480009]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.607741346488673, 3.1193612914096605]),
            {
              "landcover": 1,
              "system:index": "4"
            })])

# sentinel
bare_s = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.588761955782978, 3.110552597531067]),
            {
              "landcover": 0,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.590113789126484, 3.1101669284095776]),
            {
              "landcover": 0,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.570400011069342, 3.114240974803364]),
            {
              "landcover": 0,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.56574369621949, 3.114048140935224]),
            {
              "landcover": 0,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.566300979571633, 3.1296952794920045]),
            {
              "landcover": 0,
              "system:index": "4"
            })])

vegetation_s = ee.FeatureCollection(
        [ee.Feature(
            ee.Geometry.Point([29.61279975505381, 3.134623171579494]),
            {
              "landcover": 1,
              "system:index": "0"
            }),
        ee.Feature(
            ee.Geometry.Point([29.620696178393654, 3.1514206812868597]),
            {
              "landcover": 1,
              "system:index": "1"
            }),
        ee.Feature(
            ee.Geometry.Point([29.60730659099131, 3.15656272195781]),
            {
              "landcover": 1,
              "system:index": "2"
            }),
        ee.Feature(
            ee.Geometry.Point([29.609366527514748, 3.1041127202608414]),
            {
              "landcover": 1,
              "system:index": "3"
            }),
        ee.Feature(
            ee.Geometry.Point([29.561301341967873, 3.0982847807562592]),
            {
              "landcover": 1,
              "system:index": "4"
            })])

# TRAINING CLASSIFIERS
# landsat
training = bare.merge(vegetation)
composite1985 = ls5 \
        .filter(ee.Filter.bounds(training_region)) \
        .filter(ee.Filter.date('1985-01-01', '1986-01-01')) \
        .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
        .map(mask_ls5_clouds) \
        .select('SR_B.*') \
        .median() \
        .clip(training_region)

# overlay the point on the image to get training data
training = composite1985.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 1
})

# train a classifier
classifier_ls = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': composite1985.bandNames()
})

# sentinel
training = bare_s.merge(vegetation_s)

composite2021 = s2 \
        .filter(ee.Filter.bounds(training_region)) \
        .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(mask_s2_clouds) \
        .select('B.*') \
        .median() \
        .clip(training_region) \

# overlay the point on the image to get training data
training = composite2021.sampleRegions(**{
  'collection': training,
  'properties': ['landcover'],
  'scale': 1
})

# train a classifier
classifier_s2 = ee.Classifier.smileRandomForest(50).train(**{
  'features': training,
  'classProperty': 'landcover',
  'inputProperties': composite2021.bandNames()
})

In [7]:
# vegetation percentage change
def calculate_percentage_change(feature):
    g = feature.geometry()

    composite_ls = ls5 \
            .filter(ee.Filter.bounds(g)) \
            .filter(ee.Filter.date('1985-01-01', '1990-12-31')) \
            .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
            .map(mask_ls5_clouds) \
            .select('SR_B.*') \
            .median() \
            .clip(g)

    composite_s2 = s2 \
            .filter(ee.Filter.bounds(g)) \
            .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .map(mask_s2_clouds) \
            .select('B.*') \
            .median() \
            .clip(g)

    left_classified = composite_ls.classify(classifier_ls)
    right_classified = composite_s2.classify(classifier_s2)

    # Reclassify from 0-1 to 1-2
    left_classes = left_classified.remap([0, 1], [1, 2])
    right_classes = right_classified.remap([0, 1], [1, 2])

    initial_vegetation = left_classes.eq(2)
    bare = right_classes.eq(1)
    vegetation_to_bare = initial_vegetation.And(bare)

    area_image = vegetation_to_bare.multiply(ee.Image.pixelArea())
    area = reduce_region(area_image, ee.Reducer.sum(), g, 100)
    area_vegetation_to_bare = ee.Number(area.get('remapped')).divide(1e6)

    area_initial_vegetation = left_classified.eq(1).multiply(ee.Image.pixelArea())
    area2 = reduce_region(area_initial_vegetation, ee.Reducer.sum(), g, 100)
    area_initial_vegetation = ee.Number(area2.get('classification')).divide(1e6)

    percent_loss = area_vegetation_to_bare.divide(area_initial_vegetation).multiply(100)

    # # account for loss of vegetation from before 1985
    # #area of image
    # total_area = g.area()
    # total_SqKm = ee.Number(total_area).divide(1e6)
    # #area of bare earth 2021
    # bare_image = bare.multiply(ee.Image.pixelArea())
    # area3 = reduce_region(bare_image, ee.Reducer.sum(), g, 100)
    # area_bare = ee.Number(area3.get('remapped')).divide(1e6)
    # percent_bare = area_bare.divide(total_SqKm).multiply(100)
    
    # Show all vegetation loss areas
    Map.centerObject(feature)
    Map.addLayer(composite_ls, rgbVis_LS5, 'Composite 1985')
    Map.addLayer(composite_s2, rgbVis_S2, 'Composite 2021')
    Map.addLayer(vegetation_to_bare, {'min':0, 'max':1, 'palette': ['white', 'red']}, 'vegetation to other')
    
    # LEGEND
    legend_dict = {
        'Vegetation Loss': ('FF0000'),
    }
    Map.add_legend(legend_title='Legend', legend_dict=legend_dict, position='bottomright')
    
    print('percent loss: ', percent_loss.getInfo())
    #Map.addLayer()
    #return percent_loss.getInfo(), percent_bare.getInfo()
    #return feature.set('percent loss',  percent_loss).set('percent bare', percent_bare)
    return Map

In [8]:
calculate_percentage_change(ee.Feature(region))

percent loss:  21.808038062770386


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [ ]:
# #### IF DIVIDING INTO SQUARES - IN PROGRESS
# # CREATE SEGMENTS - FIX
# def create_segments(geometry, size):
#     # grab the top left coordinate of the bounding box
#     coords = ee.List(geometry.coordinates().get(0)).slice(0, -1)
#     top = ee.Number(ee.List(coords.get(2)).get(1))
#     left = ee.Number(ee.List(coords.get(0)).get(0))
#     top_left_point = ee.Geometry.Point([left, top])


#     # calculate square pixel width and height
#     width = int(ee.Geometry.Point(coords.get(0)).distance(ee.Geometry.Point(coords.get(1))).divide(1000 * size).getInfo()) # how many squares can fit in the width of the bounding box
#     height = int(ee.Geometry.Point(coords.get(1)).distance(ee.Geometry.Point(coords.get(2))).divide(1000 * size).getInfo()) # how many squares can fit in the height of the bounding box

#     # create a point buffer and use it to find the km distance to lat/lon degree conversion
#     buff = top_left_point.buffer(size*1000, 0.1) # create the buffer with a max % error of 0.1
#     buff_list = ee.List(buff.coordinates().get(0)) 
#     buff_length = buff_list.length()
#     right_pt = ee.List(buff_list.get(buff_length.multiply(0.75).int().subtract(1)))
#     bottom_pt = ee.List(buff_list.get(buff_length.multiply(0.5).int().subtract(1)))
#     new_lat = ee.Number(right_pt.get(0)) # given distance east of the top left point
#     new_lon = ee.Number(bottom_pt.get(1)) # given distance south of the top left point

#     diff_lon = top.subtract(new_lon) # given size converted to degrees
#     diff_lat = new_lat.subtract(left)

#     # build the list of squares
#     segments = []
#     for y in range(height + 1): # +1 to guarantee we will cover the whole region (squares may extend slightly past the bounding box)
#         left = ee.Number(ee.List(coords.get(0)).get(0))
#         for x in range(width + 1):
#             new_lat = left.add(diff_lat)
#             new_lon = top.subtract(diff_lon)

#             square = ee.Geometry.Polygon(
#                 [[[left, new_lon],
#                   [new_lat, new_lon],
#                   [new_lat, top],
#                   [left, top]]])

#             segments.append(square)

#             left = new_lat
#         top = new_lon

#     return segments

# # vegetation percentage change
# def calculate_percentage_change(feature):
#     g = feature.geometry()

#     composite_ls = ls5 \
#             .filter(ee.Filter.bounds(g)) \
#             .filter(ee.Filter.date('1985-01-01', '1990-12-31')) \
#             .filter(ee.Filter.lt('CLOUD_COVER', 20)) \
#             .map(mask_ls5_clouds) \
#             .select('SR_B.*') \
#             .median() \
#             .clip(g)

#     composite_s2 = s2 \
#             .filter(ee.Filter.bounds(g)) \
#             .filter(ee.Filter.date('2021-01-01', '2021-12-31')) \
#             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
#             .map(mask_s2_clouds) \
#             .select('B.*') \
#             .median() \
#             .clip(g)

#     left_classified = composite_ls.classify(classifier_ls)
#     right_classified = composite_s2.classify(classifier_s2)

#     # Reclassify from 0-1 to 1-2
#     left_classes = left_classified.remap([0, 1], [1, 2])
#     right_classes = right_classified.remap([0, 1], [1, 2])

#     initial_vegetation = left_classes.eq(2)
#     bare = right_classes.eq(1)
#     vegetation_to_bare = initial_vegetation.And(bare)

#     area_image = vegetation_to_bare.multiply(ee.Image.pixelArea())
#     area = reduce_region(area_image, ee.Reducer.sum(), g, 100)
#     area_vegetation_to_bare = ee.Number(area.get('remapped')).divide(1e6)

#     area_initial_vegetation = left_classified.eq(1).multiply(ee.Image.pixelArea())
#     area2 = reduce_region(area_initial_vegetation, ee.Reducer.sum(), g, 100)
#     area_initial_vegetation = ee.Number(area2.get('classification')).divide(1e6)

#     percent_loss = area_vegetation_to_bare.divide(area_initial_vegetation).multiply(100)

#     # account for loss of vegetation from before 1985
#     #area of image
#     total_area = g.area()
#     total_SqKm = ee.Number(total_area).divide(1e6)
#     #area of bare earth 2021
#     bare_image = bare.multiply(ee.Image.pixelArea())
#     area3 = reduce_region(bare_image, ee.Reducer.sum(), g, 100)
#     area_bare = ee.Number(area3.get('remapped')).divide(1e6)

#     percent_bare = area_bare.divide(total_SqKm).multiply(100)
#     print('percent loss: ', percent_loss)
#     return percent_loss
#     #return feature.set('percent loss',  percent_loss).set('percent bare', percent_bare)
    
# # Calculate values for pixresxpixres squares
# regions = create_segments(region, size)
# segments = ee.FeatureCollection(regions)
# results = segments.map(calculate_percentage_change)